In [13]:
import serial
import time
ser = serial.Serial("COM6", baudrate=9600, parity=serial.PARITY_NONE, 
                    bytesize = serial.EIGHTBITS, stopbits = serial.STOPBITS_ONE,
                    timeout = None, xonxoff = 0, rtscts = 0,)

s1range = 10000
step = 400

# ステージ駆動関数
# 400パルス = 1 degree
def s1home():
    cmd = 'H:1\r\n'
    ser.write(cmd.encode())

def s2home():
    cmd = 'H:2+\r\n'
    ser.write(cmd.encode())
        
def s1ccw(pulse):
    cmd = 'M:1+P' + str(pulse) + '\r\n'
    ser.write(cmd.encode())
    ser.write(b'G\r\n')
        
def s1cw(pulse):
    cmd = 'M:1-P' + str(pulse) + '\r\n'
    ser.write(cmd.encode())
    ser.write(b'G\r\n')
    
def s2ccw(pulse):
    cmd = 'M:2+P' + str(pulse) + '\r\n'
    ser.write(cmd.encode())
    ser.write(b'G\r\n')

def s2cw(pulse):
    cmd = 'M:2-P' + str(pulse) + '\r\n'
    ser.write(cmd.encode())
    ser.write(b'G\r\n')

def s1s2ccw(pulse):
    cmd = 'M:W+P' + str(pulse) + '+P' + str(pulse*2) + '\r\n'
    ser.write(cmd.encode())
    ser.write(b'G\r\n')
    
def currentpos(ser):
    s = ''
    while True:
        ser.write(b'Q:\r\n')
        c = ser.read(1)
        # print(c)
        s = s+c.decode('UTF-8')
        if c == b'\n': 
            break
    print(s)
    ssplit = s.split(',')
    return int(ssplit[0])

def sprmeas():
    s1pos = currentpos(ser)
    s1start = s1pos# s1pos : theta1 current pulse position
    s1endpos = s1start + s1range
    
    while s1pos < s1endpos:
        # power measurement
        # 計測が完了するまでwaitしておく必要があるだろう
        print(str(s1pos)+'\r\n')
        #with open(path, mode='w') as f:    
        #    f.write(str(pos)+'\r\n')
        # ファイルへのデータ書き込み、大下君に聞いてください。記載するべき事項は、角度+強度値です。
        s1s2ccw(step)
        time.sleep(0.5)
        s1pos = s1pos + step
    ser.close()

In [15]:
s2cw(10000)

In [12]:
ser.close()